# 07 - ECDSA: Цифровая подпись (CRYPTO-11)

Этот notebook покрывает пошаговую реализацию ECDSA, верификацию подписей
и демонстрацию атаки повторного использования nonce.

**Что вы изучите:**
1. Ручная реализация подписи ECDSA по формулам
2. Ручная верификация подписи ECDSA
3. Подпись через библиотеку ecdsa (SigningKey API)
4. **АТАКА**: повторное использование nonce (извлечение приватного ключа)
5. Детерминированная подпись RFC 6979
6. Восстановление публичного ключа из подписи

**Используемые библиотеки:** `ecdsa`, `hashlib`, стандартная библиотека Python

In [ ]:
import hashlib
import os
from ecdsa import SECP256k1, SigningKey, VerifyingKey, BadSignatureError
from ecdsa.numbertheory import inverse_mod
from ecdsa.ellipticcurve import Point

# Параметры кривой secp256k1
curve = SECP256k1
G = curve.generator
n = curve.order

print(f"Кривая: secp256k1")
print(f"Порядок группы n: {n}")
print(f"Генератор G.x: {G.x()}")
print(f"Генератор G.y: {G.y()}")
print(f"Размер ключа: {curve.baselen} байт")

---
## Часть 1: Пошаговая подпись ECDSA

Реализуем ECDSA подпись вручную, используя формулы:

1. Выбираем случайный k (nonce)
2. R = k * G
3. r = R.x mod n
4. s = k^(-1) * (h + r * d) mod n
5. Подпись: (r, s)

In [ ]:
# Шаг 1: Генерация ключей
d = int.from_bytes(os.urandom(32), 'big') % (n - 1) + 1  # приватный ключ
Q = d * G  # публичный ключ (точка на кривой)

print(f"Приватный ключ d: {hex(d)[:20]}...")
print(f"Публичный ключ Q.x: {hex(Q.x())[:20]}...")
print(f"Публичный ключ Q.y: {hex(Q.y())[:20]}...")

In [ ]:
# Шаг 2: Хеш сообщения
message = b"Transfer 1 BTC from Alice to Bob"
h = int(hashlib.sha256(message).hexdigest(), 16)

print(f"Сообщение: {message.decode()}")
print(f"SHA-256 хеш h: {hex(h)[:20]}...")
print(f"Длина хеша: {h.bit_length()} бит")

In [ ]:
# Шаг 3: Подпись ECDSA пошагово

# 3.1 Выбираем случайный nonce k
k = int.from_bytes(os.urandom(32), 'big') % (n - 1) + 1
print(f"Шаг 1: Nonce k = {hex(k)[:20]}...")

# 3.2 Вычисляем точку R = k * G
R = k * G
print(f"Шаг 2: R = k*G")
print(f"  R.x = {hex(R.x())[:20]}...")
print(f"  R.y = {hex(R.y())[:20]}...")

# 3.3 Вычисляем r = R.x mod n
r = R.x() % n
print(f"Шаг 3: r = R.x mod n = {hex(r)[:20]}...")

# 3.4 Вычисляем s = k^(-1) * (h + r*d) mod n
k_inv = inverse_mod(k, n)
s = (k_inv * (h + r * d)) % n
print(f"Шаг 4: k^(-1) mod n = {hex(k_inv)[:20]}...")
print(f"  h + r*d mod n = {hex((h + r * d) % n)[:20]}...")
print(f"  s = k^(-1) * (h + r*d) mod n = {hex(s)[:20]}...")

# 3.5 Подпись
print(f"\nПодпись ECDSA:")
print(f"  r = {hex(r)}")
print(f"  s = {hex(s)}")
print(f"  Размер: 2 * 32 = 64 байта")

---
## Часть 2: Пошаговая верификация ECDSA

Алгоритм верификации:
1. s_inv = s^(-1) mod n
2. u1 = h * s_inv mod n
3. u2 = r * s_inv mod n
4. P = u1 * G + u2 * Q
5. Проверяем: P.x mod n == r ?

In [ ]:
# Верификация ECDSA пошагово

# 1. Вычисляем обратный к s
s_inv = inverse_mod(s, n)
print(f"Шаг 1: s^(-1) mod n = {hex(s_inv)[:20]}...")

# 2. u1 = h * s^(-1) mod n
u1 = (h * s_inv) % n
print(f"Шаг 2a: u1 = h * s^(-1) mod n = {hex(u1)[:20]}...")

# 3. u2 = r * s^(-1) mod n
u2 = (r * s_inv) % n
print(f"Шаг 2b: u2 = r * s^(-1) mod n = {hex(u2)[:20]}...")

# 4. P = u1 * G + u2 * Q
P = u1 * G + u2 * Q
print(f"Шаг 3: P = u1*G + u2*Q")
print(f"  P.x = {hex(P.x())[:20]}...")

# 5. Проверяем P.x mod n == r
check = P.x() % n
valid = check == r
print(f"\nШаг 4: P.x mod n = {hex(check)[:20]}...")
print(f"        r        = {hex(r)[:20]}...")
print(f"\nПодпись валидна: {valid}")

In [ ]:
# Проверка с измененным сообщением (подпись должна быть невалидна)
fake_message = b"Transfer 100 BTC from Alice to Bob"  # изменили сумму!
h_fake = int(hashlib.sha256(fake_message).hexdigest(), 16)

# Пересчитываем с поддельным хешем
u1_fake = (h_fake * s_inv) % n
u2_fake = (r * s_inv) % n  # u2 не зависит от h
P_fake = u1_fake * G + u2_fake * Q

valid_fake = P_fake.x() % n == r
print(f"Оригинальное сообщение: подпись валидна = {valid}")
print(f"Поддельное сообщение:   подпись валидна = {valid_fake}")
print(f"\nПодпись привязана к конкретному сообщению!")

---
## Часть 3: Подпись через библиотеку ecdsa

Использование высокоуровневого API для подписи и верификации.

In [ ]:
# Высокоуровневое API
sk = SigningKey.generate(curve=SECP256k1)
vk = sk.get_verifying_key()

message = b"Transfer 1 BTC to Alice"

# Подпись
signature = sk.sign(message)
print(f"Подпись (hex): {signature.hex()[:40]}...")
print(f"Размер: {len(signature)} байт")

# Верификация
try:
    vk.verify(signature, message)
    print("Подпись верифицирована!")
except BadSignatureError:
    print("Подпись невалидна!")

# Попытка верификации с другим сообщением
try:
    vk.verify(signature, b"Transfer 100 BTC to Alice")
    print("Подпись верифицирована для поддельного сообщения (ОШИБКА!)")
except BadSignatureError:
    print("Подпись отклонена для поддельного сообщения (корректно!)")

In [ ]:
# Каждая подпись ECDSA разная (случайный k)
sig1 = sk.sign(message)
sig2 = sk.sign(message)
sig3 = sk.sign(message)

print(f"sig1: {sig1.hex()[:32]}...")
print(f"sig2: {sig2.hex()[:32]}...")
print(f"sig3: {sig3.hex()[:32]}...")
print(f"\nВсе подписи различны: {sig1 != sig2 and sig2 != sig3}")
print("Потому что каждый раз используется новый случайный k")

# Но все три валидны!
for i, sig in enumerate([sig1, sig2, sig3], 1):
    vk.verify(sig, message)
    print(f"sig{i} валидна: True")

---
## Часть 4: АТАКА повторного использования nonce

### *** ВНИМАНИЕ: ОБРАЗОВАТЕЛЬНЫЙ МАТЕРИАЛ ***

Этот раздел демонстрирует, почему НИКОГДА нельзя использовать один и тот же nonce k
для двух разных подписей. Повторное использование k позволяет **извлечь приватный ключ**.

**Реальный пример:** В 2010 году группа fail0verflow извлекла приватный ключ Sony
из подписей прошивок PlayStation 3. Sony использовала k = 4 (константу!) для всех подписей.

### *** НИКОГДА не используйте фиксированный nonce в реальных приложениях! ***

In [ ]:
# ================================================================
# *** ТОЛЬКО ДЛЯ ОБУЧЕНИЯ! ***
# *** НИКОГДА не используйте фиксированный k в реальном коде! ***
# ================================================================

# Генерируем ключевую пару
d_victim = int.from_bytes(os.urandom(32), 'big') % (n - 1) + 1
Q_victim = d_victim * G
print(f"Приватный ключ жертвы: {hex(d_victim)[:20]}...")
print(f"(Этот ключ мы восстановим из подписей!)\n")

# Два разных сообщения
m1 = b"Transfer 1 BTC to Alice"
m2 = b"Transfer 2 BTC to Bob"
h1 = int(hashlib.sha256(m1).hexdigest(), 16)
h2 = int(hashlib.sha256(m2).hexdigest(), 16)

# *** ОШИБКА: используем ОДИНАКОВЫЙ nonce k для обеих подписей ***
k_reused = int.from_bytes(os.urandom(32), 'big') % (n - 1) + 1

# Подпись 1
R_1 = k_reused * G
r_1 = R_1.x() % n
s_1 = (inverse_mod(k_reused, n) * (h1 + r_1 * d_victim)) % n

# Подпись 2 (с ТЕМ ЖЕ k!)
R_2 = k_reused * G  # R будет тот же!
r_2 = R_2.x() % n   # и r тоже!
s_2 = (inverse_mod(k_reused, n) * (h2 + r_2 * d_victim)) % n

print(f"Подпись 1: r = {hex(r_1)[:20]}..., s = {hex(s_1)[:20]}...")
print(f"Подпись 2: r = {hex(r_2)[:20]}..., s = {hex(s_2)[:20]}...")
print(f"\nr одинаковое в обеих подписях: {r_1 == r_2}")
print(f"s разное (разные сообщения): {s_1 != s_2}")
print(f"\nОДИНАКОВОЕ r -- это красный флаг! Значит k повторяется!")

In [ ]:
# ================================================================
# АТАКА: извлечение приватного ключа
# ================================================================

# Атакующий видит: (r, s1, h1) и (r, s2, h2) -- r одинаковое!

# Шаг 1: Вычисляем k
# Из формул: s1 = k^(-1)(h1 + r*d), s2 = k^(-1)(h2 + r*d)
# Вычитаем: s1 - s2 = k^(-1)(h1 - h2)
# Значит:   k = (h1 - h2) * (s1 - s2)^(-1) mod n

k_recovered = ((h1 - h2) * inverse_mod(s_1 - s_2, n)) % n
print(f"Шаг 1: Восстановленный k = {hex(k_recovered)[:20]}...")
print(f"Оригинальный k          = {hex(k_reused)[:20]}...")
print(f"Совпадают: {k_recovered == k_reused}\n")

# Шаг 2: Вычисляем приватный ключ d
# Из формулы: s = k^(-1)(h + r*d)
# Значит:     s*k = h + r*d
# Значит:     d = (s*k - h) * r^(-1) mod n

d_recovered = ((s_1 * k_recovered - h1) * inverse_mod(r_1, n)) % n
print(f"Шаг 2: Восстановленный d = {hex(d_recovered)[:20]}...")
print(f"Оригинальный d          = {hex(d_victim)[:20]}...")
print(f"Совпадают: {d_recovered == d_victim}")

if d_recovered == d_victim:
    print(f"\n*** ПРИВАТНЫЙ КЛЮЧ ПОЛНОСТЬЮ ВОССТАНОВЛЕН! ***")
    print(f"*** Атакующий может подписывать любые транзакции! ***")

In [ ]:
# Проверка: атакующий может создать подпись от имени жертвы

malicious_message = b"Transfer all BTC to attacker_address"
h_mal = int(hashlib.sha256(malicious_message).hexdigest(), 16)

# Атакующий подписывает используя украденный ключ
k_new = int.from_bytes(os.urandom(32), 'big') % (n - 1) + 1
R_mal = k_new * G
r_mal = R_mal.x() % n
s_mal = (inverse_mod(k_new, n) * (h_mal + r_mal * d_recovered)) % n

# Верификация: эта подпись валидна для публичного ключа жертвы!
s_inv_mal = inverse_mod(s_mal, n)
u1_mal = (h_mal * s_inv_mal) % n
u2_mal = (r_mal * s_inv_mal) % n
P_mal = u1_mal * G + u2_mal * Q_victim

forged_valid = P_mal.x() % n == r_mal
print(f"Сообщение от атакующего: {malicious_message.decode()}")
print(f"Поддельная подпись валидна для ключа жертвы: {forged_valid}")
print(f"\n*** Это показывает, почему повторное использование nonce ***")
print(f"*** является КАТАСТРОФИЧЕСКОЙ уязвимостью! ***")

---
## Часть 5: Детерминированная подпись RFC 6979

RFC 6979 решает проблему случайного nonce, вычисляя k детерминированно:

```
k = HMAC-DRBG(private_key, message_hash)
```

Одинаковый ключ + сообщение всегда дают одинаковую подпись.

In [ ]:
# Детерминированная подпись (RFC 6979)
sk = SigningKey.generate(curve=SECP256k1)
vk = sk.get_verifying_key()

message = b"Transfer 1 BTC to Alice"

# sign_deterministic использует RFC 6979 для вычисления k
sig_det_1 = sk.sign_deterministic(message)
sig_det_2 = sk.sign_deterministic(message)
sig_det_3 = sk.sign_deterministic(message)

print(f"sig1: {sig_det_1.hex()[:32]}...")
print(f"sig2: {sig_det_2.hex()[:32]}...")
print(f"sig3: {sig_det_3.hex()[:32]}...")
print(f"\nВсе подписи одинаковы: {sig_det_1 == sig_det_2 == sig_det_3}")
print("Детерминированный k -> одинаковая подпись для одного сообщения")

# Разные сообщения -> разные k -> разные подписи
sig_other = sk.sign_deterministic(b"Transfer 2 BTC to Bob")
print(f"\nДругое сообщение:")
print(f"sig_other: {sig_other.hex()[:32]}...")
print(f"Подписи разные: {sig_det_1 != sig_other}")

# Верификация
vk.verify(sig_det_1, message)
print(f"\nДетерминированная подпись верифицирована!")

In [ ]:
# Сравнение: случайная vs детерминированная подпись
print("Случайная подпись (sk.sign):")
sigs_random = [sk.sign(message).hex()[:24] for _ in range(5)]
for i, s in enumerate(sigs_random, 1):
    print(f"  sig{i}: {s}...")
print(f"  Все уникальны: {len(set(sigs_random)) == 5}")

print(f"\nДетерминированная подпись (sk.sign_deterministic):")
sigs_det = [sk.sign_deterministic(message).hex()[:24] for _ in range(5)]
for i, s in enumerate(sigs_det, 1):
    print(f"  sig{i}: {s}...")
print(f"  Все идентичны: {len(set(sigs_det)) == 1}")

print(f"\nРекомендация: ВСЕГДА используйте sign_deterministic() в продакшене!")

---
## Часть 6: Использование random_k для пошаговой визуализации

Библиотека ecdsa позволяет передать свой генератор k через параметр `k`.
Это полезно для обучения, но **НИКОГДА** не используйте предсказуемый k в продакшене.

In [ ]:
# ================================================================
# *** ТОЛЬКО ДЛЯ ОБУЧЕНИЯ! ***
# *** НИКОГДА не используйте фиксированный k в реальном коде! ***
# ================================================================

sk = SigningKey.generate(curve=SECP256k1)
vk = sk.get_verifying_key()

message = b"Educational demo"

# Фиксированный k для демонстрации (через random_k функцию)
fixed_k = 12345678901234567890  # *** НИКОГДА ТАК НЕ ДЕЛАЙТЕ! ***

sig_fixed_1 = sk.sign(message, k=fixed_k)
sig_fixed_2 = sk.sign(message, k=fixed_k)

print(f"Подпись с фиксированным k:")
print(f"sig1: {sig_fixed_1.hex()[:32]}...")
print(f"sig2: {sig_fixed_2.hex()[:32]}...")
print(f"Совпадают: {sig_fixed_1 == sig_fixed_2}")
print(f"\n*** Это только для обучения! В продакшене: sign_deterministic() ***")

---
## Упражнения

### Упражнение 1: Подпись и верификация

Создайте ключевую пару, подпишите сообщение "My first ECDSA signature" и верифицируйте.
Затем измените один байт в подписи и покажите, что верификация провалится.

In [ ]:
# Упражнение 1: ваш код здесь

# sk = SigningKey.generate(curve=SECP256k1)
# vk = sk.get_verifying_key()
# message = b"My first ECDSA signature"
# sig = sk.sign_deterministic(message)
# 
# # Верификация оригинальной подписи
# ...
# 
# # Изменяем один байт
# tampered = bytearray(sig)
# tampered[0] = (tampered[0] + 1) % 256
# tampered = bytes(tampered)
# 
# # Пробуем верифицировать измененную подпись
# ...

### Упражнение 2: Восстановление публичного ключа из подписи

В Ethereum `ecrecover` восстанавливает публичный ключ из подписи (v, r, s) и хеша.
Реализуйте восстановление вручную:

1. Из r восстановите точку R (есть два варианта R с одинаковым x)
2. Вычислите Q = r^(-1) * (s*R - h*G)

In [ ]:
# Упражнение 2: ваш код здесь

# Подсказка: для восстановления R из r нужно решить y^2 = r^3 + 7 (mod p)
# Для secp256k1: p mod 4 = 3, поэтому y = (r^3 + 7)^((p+1)/4) mod p

# sk = SigningKey.generate(curve=SECP256k1)
# ...
# Q_recovered = ...
# assert Q_recovered == Q_original

---
## Челлендж: Low-s нормализация (Bitcoin)

В Bitcoin подписи ECDSA нормализуются: если s > n/2, заменяем s на n - s.
Это предотвращает transaction malleability.

**Задача:**
1. Подпишите сообщение
2. Проверьте, является ли s > n/2
3. Если да, замените s на n - s
4. Убедитесь, что нормализованная подпись всё ещё валидна

In [ ]:
# Челлендж: Low-s нормализация

# sk = SigningKey.generate(curve=SECP256k1)
# vk = sk.get_verifying_key()
# n = SECP256k1.order
# 
# # Подписываем несколько раз, пока не получим high-s подпись
# for i in range(100):
#     sig = sk.sign(b"test")
#     r_val = int.from_bytes(sig[:32], 'big')
#     s_val = int.from_bytes(sig[32:], 'big')
#     if s_val > n // 2:
#         print(f"Найдена high-s подпись на итерации {i}")
#         s_low = n - s_val
#         # Создаем нормализованную подпись
#         sig_normalized = r_val.to_bytes(32, 'big') + s_low.to_bytes(32, 'big')
#         # Проверяем валидность
#         ...
#         break